# Translating French Incidents

In [2]:
import csv
import pandas
import time
from spacy.lang.en.stop_words import STOP_WORDS
from googletrans import Translator
translator = Translator()
pandas.set_option('display.max_colwidth', 0)

In [3]:
combined = pandas.read_csv('03 Combined.csv', delimiter=',', encoding='latin-1').fillna('')
combined['Incident Description'] = combined['Incident Description'].apply(lambda x:x.replace('\r', '.').replace('\n', '.'))
combined[:1]

,ID,Process Step,Problem Type,Contributing Factors,Overall Severity,Incident Description
0,2511,Treatment delivery,"Wrong, missing, mislabeled, or damaged treatment accessories",Distraction or diversions involving staff,None,Non prescribed bolus . Bolus was not prescribed by MD. Pt received 2 treatments wothout bolus.


In [5]:
translations = pandas.read_csv('JK_translations.csv', delimiter=',', encoding='latin-1').fillna('')
translations

,Orginal French,JK Translate,Google Translate,Comment
0,"Patiente traitÃ©e au sein et Ã la rÃ©gion de l'axillio-sus-claviculaire par une nouvelle technique de traitement avec MLC (multi leaf collimator) de type ""sliding window"". Cette technique a Ã©tÃ© mise en utilisation clinique avant d'avoir Ã©tÃ© validÃ©e par des mesures complÃ¨tes Ã l'appareil de traitement. Une mesure dosimÃ©trie de la jonction spÃ©cifique Ã cette.patiente a Ã©tÃ© acquise avant le dÃ©but de ses traitements. La mesure dosimÃ©trique de la jonction a dÃ©montrÃ© une non conformitÃ© du plan de traitement qui a dÃ» Ãªtre retravaillÃ©. De.nouvelles mesures ont finalement confirmÃ© la conformitÃ© du plan de traitement retravaillÃ©.","Patient treated at the breast and the axillia-supraclavicular reguion by a new technique with MLC (multi-leaf collimator) of type ""sliding window"". This technique was implemented in the clinic without having been validated by complete measurements at the treatment machine. A dosimetry measurement at the specific junction of this patient was acquired prior to starting her treatments. The dosimetric measurement showed a non-conformity with the treatment plan, whcih had to be re-worked. New measurements finally confirmed the conformity of the re-worked treeatment plan","Patient treated within and ?? the region of axillio-supraclavicular by a new technique of treatment with MLC (multi leaf collimator) type ""sliding window"". This technique was ?? put into clinical use before having been ?? validated by comprehensive measurements ?? the treatment apparatus. A dosimetry measurement of the specific junction? this patient was ?? acquired before the start of his treatments. The dosimetric measurement of the junction has demonstrated a nonconformity ?? of the treatment plan that has To be reworked. New measures finally confirmed ?? conformity? of the reworked treatment plan.","Google translate left out the word ""breast"", which is a major omission. Otherwise, the translation is ok"
1,"2017-03-21 : Suite la consultation mdicale, le radio-oncologue cre une tche de suivi adresse aux secrtaires (.RO Suivi des rsultats dexamen) au lieu dune tche adresse lui-mme (.RO Demande de tx en attente). Le suivi demandait dattendre le retour tlphonique d'un chirurgien. Ce retour dappel est un pralable la mise en traitement du patient. Les secrtaires qui ont reues la tche s'attendaient recevoir un appel du chirurgien. La date butoir de la tche a t reporte de semaine en semaine par les secrtaires n'ayant aucun retour d'appel du chirurgien..Deux trois semaines aprs la consultation, le radio-oncologue a reu le retour dappel du chirurgien. Sattendant tre relanc par les secrtaires afin deffectuer un suivi sur leur tche, le radio-oncologue na fait aucune action suite lappel du chirurgien..2017-05-15 : Inquiet du long dlai suite la consultation, le patient appelle son infirmire pivot pour savoir quand dbutera ses traitements, rappelant quil doit recevoir cinq traitements de radiothrapie avant sa chirurgie. Le radio-oncologue est avis de l'appel. Il envoie immdiatement la tche pour la programmation des rendez-vous de planification et des traitements l'agente administrative.","Following the medical consultation, the radiation oncologist created a follow-up task addressed to the secretaries (RO follow-up exam results) instead of a task addressed to him/herself (RO request for treatment in waiting). The follow-up asked to wait for a telephone reply of a surgeon. This reply call is a prerequesite to the starting of treatment of the paitent. The secretaries received the task that was waiting for the call of the surgeon. The target date of the task was postponed week-by-week by the secretaries as they had not received the reply call of the surgeon. Two to three weeks after the consultation, the radiation oncologist received the reply call of the surgeon.","2017-03-21: Continued ?? medical consultation, the radiation oncologist creates a follow-up task addressed to th

In [30]:
# Compile list of stop words with more than 2 characters.
stop_words = [word for word in STOP_WORDS if len(word)>2]

In [31]:
# Returns True or False
def has_english_stop_words(string):
    return len([word for word in set(string.lower().split()) if word in stop_words])>0

In [32]:
# Detect wheterh incident descriptions are in English or French
combined['Language'] = ''
for i in range(len(combined)):
    incident_description = combined.at[i, 'Incident Description']
    if has_english_stop_words(incident_description):
        combined.at[i, 'Language'] = 'En'
    else:
        if translator.detect(incident_description).lang == 'en':
            combined.at[i, 'Language'] = 'En'
        else:
            combined.at[i, 'Language'] = 'Fr'

AttributeError: 'NoneType' object has no attribute 'group'

In [6]:
def translate(string):
    while True:
        try:
            en = translator.translate(string).text
            print(en)
            return en
        except Exception:
            time.sleep(1)

In [7]:
# If the original incident was in English, then 
combined['Translated'] = ''
for i in range(len(combined)):
    incident_description = combined.at[i, 'Incident Description']
    language = combined.at[i, 'Language']
    if language == 'En':
        combined.at[i, 'Translated'] = incident_description
    else:
        translation = translate(incident_description)
        combined.at[i, 'Translated'] = translation

FALL. As a result of a loss of balance coming out of the CT-scan room. the patient fell on his back by hitting his buttocks first on the ground and head to hit the ground afterwards.
tesdtrewre
Patient treated at breast and ?? the region of axillio-supraclavicular by a new technique of treatment with MLC (multi leaf collimator) type "sliding window". This technique has been ?? put into clinical use before having to ?? validated by comprehensive measurements Ã ?? the treatment apparatus. A dosimetric measurement of the specific junction Ã ?? This patient has to ?? acquired before the end of his treatments. The dosimetric measurement of the junction has been shown ?? a nonconformity of the treatment plan that has fallen away? To be reworked. New measures finally confirmed ?? conformity? of the reworked treatment plan.
2017-03-21: Following Ã ?? the medical consultation, the radiation oncologist creates a follow-up task addressed to the secretaries (.RO Follow-up review results) instead o

In [16]:
combined[:1]

,ID,Process Step,Problem Type,Contributing Factors,Overall Severity,Incident Description
0,2511,Treatment delivery,"Wrong, missing, mislabeled, or damaged treatment accessories",Distraction or diversions involving staff,None,Non prescribed bolus . Bolus was not prescribed by MD. Pt received 2 treatments wothout bolus.


In [17]:
combined['Language']

KeyError: 'Language'

In [9]:
combined.to_csv('04 Combined Translated.csv', encoding='utf-8', index=False, quoting=csv.QUOTE_NONNUMERIC)